## a) Vorwärtskinematik


In [63]:
import numpy as mp
from math import sin, cos, sqrt, atan2
from transformations import *

In [76]:
# setup of variables
x_r = 2
y_r = 1
r = 0.1
theta = np.radians(30)

In [77]:
TL = trans(np.array((x_r, y_r, r)).transpose())
TL

array([[1. , 0. , 0. , 2. ],
       [0. , 1. , 0. , 1. ],
       [0. , 0. , 1. , 0.1],
       [0. , 0. , 0. , 1. ]])

In [78]:
Rz = rot2trans(rotz(theta))
Rz

array([[ 0.8660254, -0.5      ,  0.       ,  0.       ],
       [ 0.5      ,  0.8660254,  0.       ,  0.       ],
       [ 0.       ,  0.       ,  1.       ,  0.       ],
       [ 0.       ,  0.       ,  0.       ,  1.       ]])

In [92]:
T_OR = np.matmul(trans(np.array((x_r, y_r,r)).transpose()), rot2trans(rotz(theta)))
T_OR

array([[ 0.8660254, -0.5      ,  0.       ,  2.       ],
       [ 0.5      ,  0.8660254,  0.       ,  1.       ],
       [ 0.       ,  0.       ,  1.       ,  0.1      ],
       [ 0.       ,  0.       ,  0.       ,  1.       ]])

In [80]:
l = 0.6
h = 0.2

In [93]:
T_R_DB = trans(np.array((l/2-a/2, 0,h)).transpose())
T_R_DB

array([[1. , 0. , 0. , 0.3],
       [0. , 1. , 0. , 0. ],
       [0. , 0. , 1. , 0.2],
       [0. , 0. , 0. , 1. ]])

In [94]:
alpha = np.radians(40)
b = 0 # simplification! was 0.1

In [95]:
T_DB_D = np.matmul(trans(np.array((0, 0, b/2)).transpose()),
                   np.matmul(rot2trans(rotz(alpha)), rot2trans(rotx(np.radians(90)))))
T_DB_D

array([[ 7.66044443e-01, -3.93593894e-17,  6.42787610e-01,
         0.00000000e+00],
       [ 6.42787610e-01,  4.69066938e-17, -7.66044443e-01,
         0.00000000e+00],
       [ 0.00000000e+00,  1.00000000e+00,  6.12323400e-17,
         0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00]])

In [96]:
a = 0 #  simplification, was 0.1
l_1 = 0.5
betha_1 = np.radians(30)
l_2 = 0.5
betha_2 = np.radians(-10)

In [97]:
T_D_A1 = np.matmul(trans(np.array((0, 0, a/2)).transpose())
    ,np.matmul(rot2trans(rotz(betha_1)), 
                   trans(np.array((l_1, 0, 0)).transpose())))
T_D_A1

array([[ 0.8660254, -0.5      ,  0.       ,  0.4330127],
       [ 0.5      ,  0.8660254,  0.       ,  0.25     ],
       [ 0.       ,  0.       ,  1.       ,  0.       ],
       [ 0.       ,  0.       ,  0.       ,  1.       ]])

In [98]:
T_A1_A2 = np.matmul(rot2trans(rotz(betha_2)),
                   trans(np.array((l_2, 0, 0)).transpose())) 
T_A1_A2

array([[ 0.98480775,  0.17364818,  0.        ,  0.49240388],
       [-0.17364818,  0.98480775,  0.        , -0.08682409],
       [ 0.        ,  0.        ,  1.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [99]:
p_A2 = np.array((0, 0, 0, 1)).transpose()

In [100]:
p_O = np.matmul(T_OR, 
                np.matmul(T_R_DB, 
                    np.matmul(T_DB_D ,
                        np.matmul(T_D_A1,
                              np.matmul(T_A1_A2, p_A2)))))
p_O

array([2.56860359, 1.99840995, 0.72101007, 1.        ])

In [101]:
#p_O =[2.57228695 1.95630894 0.77101007 1. ]   # Lösung von Prof. Blaich

p_R =
 [0.99163013 0.58034659 0.62101007 1.        ]
from Michael Blaich to everyone:    10:57 AM
p_O =
 [2.57228695 1.95630894 0.77101007 1.        ]

## Inverse Kinematik

In [102]:
p_R = np.matmul(T_R_DB,np.matmul(T_DB_D, np.matmul(T_D_A1, np.matmul(T_A1_A2, p_A2))))
p_R

array([0.99163013, 0.58034659, 0.62101007, 1.        ])

In [103]:
p_D = np.matmul(T_D_A1, np.matmul(T_A1_A2, p_A2))
p_D

array([0.90285901, 0.42101007, 0.        , 1.        ])

In [104]:
x_D = p_D[0]
x_D

0.9028590122851736

In [105]:
y_D = p_D[1]
y_D

0.42101007166283433

In [106]:
z_D = p_R[2] # Seitenansicht!
z_D

0.6210100716628344

In [107]:
a = sqrt(x_D**2 + z_D**2)
a

1.0958138095366559

In [108]:
c = (a**2 - l_1**2 - l_2**2)/2*l_1
c

0.1752019762928096

In [109]:
epsilon = -1 # elbow up

In [110]:
b = epsilon * sqrt(l_2**2 - c**2)
b

-0.468299335364779

In [111]:
betha_1 =atan2(z_D, x_D) - atan2(b, (l_1 + c))
betha_1, np.degrees(betha_1)

(1.2089057882280811, 69.26519949440512)

In [112]:
betha_2 = atan2(b, c)
betha_2, np.degrees(betha_2)

(-1.2127939605121516, -69.4879753563021)

In [113]:
alpha = atan2(y_D, x_D)
alpha, np.degrees(alpha)

(0.43633231299858233, 24.999999999999996)

In [114]:
def calculate_KS_Denavit_Hartenberg(d, theta, a, alpha):
    """
    Calculate the Transformation of one KS to another for the robot arms with Denavit-Hartenberg parameters
    :param d: shift along the old z-axis so that it lies on the new joint axis
    :param theta: rotate around the z-axis so that the old x-axis is in right angle to the new joint axis
    :param a: shift along the old x-axis until the origin is on the new joint axis
    :param alpha: rotate around the new x-axis around alpha so that the z-axis lies on the new joint axis
    :return: transformation
    """
    theta = np.radians(theta)   # my implementation is in radians
    alpha = np.radians(alpha)   # my implementation is in radians
    translation_d = trans((0, 0, d))
    rotate_theta = rot2trans(rotz(theta))
    translation_a = trans((a, 0, 0))
    rotate_alpha = rot2trans(rotx(alpha))
    transformation_ksx_to_ksy = np.matmul(np.matmul(np.matmul(translation_d, rotate_theta), translation_a),
                                          rotate_alpha)
    return transformation_ksx_to_ksy

In [115]:
def forward_kinematic(alpha, betha_1, betha_2):
    r = 0.1
    l = 0.6
    h = 0.2
    a = 0 # simplification, was 0.1
    b = 0 # simplification, was 0.1
    l_1 = 0.5
    l_2 = 0.5
    p_A2 = np.array((0, 0, 0, 1)).transpose()
    print(alpha, np.degrees(alpha))
    print(betha_1, np.degrees(betha_1))
    print(betha_2, np.degrees(betha_2))
    
    T_R_DB = trans(np.array((l/2-r, 0,h)).transpose())
    T_DB_D = np.matmul(trans(np.array((0, 0, b/2)).transpose()),
                   np.matmul(rot2trans(rotz(alpha)), rot2trans(rotx(np.radians(90)))))
    T_D_A1 = np.matmul(trans(np.array((0, 0, a/2)).transpose()),
                       np.matmul(rot2trans(rotz(betha_1)), 
                           trans(np.array((l_1, 0, 0)).transpose())))
    T_A1_A2 = np.matmul(rot2trans(rotz(betha_2)),
                   trans(np.array((l_2, 0, 0)).transpose())) 
    p_R = np.matmul(T_R_DB,np.matmul(T_DB_D, np.matmul(T_D_A1, np.matmul(T_A1_A2, p_A2))))
    return p_R
    

In [116]:
# forward_kinematic(alpha, betha_1, betha_2) # before the simplification
#array([0.92376951, 0.54204436, 0.67101007, 1.        ])

In [117]:
forward_kinematic(np.radians(40), np.radians(30), np.radians(-10))

0.6981317007977318 40.0
0.5235987755982988 29.999999999999996
-0.17453292519943295 -10.0


array([0.89163013, 0.58034659, 0.62101007, 1.        ])

In [118]:
forward_kinematic(alpha, betha_1, betha_2)

0.43633231299858233 24.999999999999996
1.2089057882280811 69.26519949440512
-1.2127939605121516 -69.4879753563021


array([0.81358641, 0.28612004, 0.6656705 , 1.        ])

In [119]:
def backward_kinematic(p_A2):
    # point in relation to robot coordinate system
    p_R = np.matmul(T_R_DB,np.matmul(T_DB_D, np.matmul(T_D_A1, np.matmul(T_A1_A2, p_A2))))
    # point in relation to "Drehteller"
    p_D = np.matmul(T_D_A1, np.matmul(T_A1_A2, p_A2))
    x_D = p_D[0]
    y_D = p_D[1]
    x_R = p_R[0]
    y_R = p_R[1]
    
    a = sqrt(x_D**2 + y_D**2)
    c = (a**2 - l_1**2 - l_2**2)/2*l_1
    epsilon = -1 # elbow up
    betha_1 =atan2(z_D, x_D) - atan2(b, (l_1 + c))
    betha_2 = atan2(b, c)
    alpha = atan2(y_R, x_R)
    print("alpha " + str(alpha))
    print("betha_1 " + str(betha_1))
    print("betha_2 " + str(betha_2))
    return alpha, betha_1, betha_2

In [120]:
b_alpha, b_betha_1, b_betha_2 = backward_kinematic(p_A2)


alpha 0.5294995915107659
betha_1 1.2470091805894987
betha_2 -1.3137436316137807


In [121]:
forward_kinematic(b_alpha, b_betha_1, b_betha_2)

0.5294995915107659 30.338091847467997
1.2470091805894987 71.44836306184537
-1.3137436316137807 -75.27196545365923


array([0.76786464, 0.33233995, 0.64067619, 1.        ])